# Molecular Dynamics and Monte Carlo
In quantum chemical studies, we usually focus on computing the properties of a molecule using one or a few structures. Typically, a geometry optimization is carried out in order to determine the minimum energy structure, and then the properties of the molecule are computed at the minimum or minima. This procedure is acceptable for small or rigid molecules which do not have a lot of structural flexibility. However, for larger molecules or for a big collection of small molecules, there will be numerous low energy structures that are accessible under typical conditions (e.g. ambient temperature). To get a more appropriate description of such systems, we need to explore their various possible configurations and obtain some *average* picture. Monte Carlo (MC) and molecular dynamics (MD) methods enable the exploration of the various possible configurations of the system.

## Overview of Monte Carlo and Molecular Dynamics Methods
MC and MD methods are powerful techniques for sampling the possible configurations of the system. The state of the system is completely specified by the positions and momenta of all the particles. The *phase space* is the space in which all possible states of the system (positions and momenta) are represented. 

In MC methods, we start from some initial structure of the system. Then, we *randomly* perturb the coordinates of a randomly chosen particle. In the popular *Metropolis* approach, we always accept the move if the energy of the system decreases. If the energy increases, we generate a random number between 0 and 1 and accept the move if $e^{-\Delta E/kT}$ is larger than the random number. This acceptance and rejection criteria allows overcoming potential energy barriers and the escape from local minima. Typically, the perturbation should be small so as to have a reasonable acceptance ration. However, this means that only a small portion of the configurational space can be explored. The Metropolis algorithm depends on the temperature and therefore MC methods are normally perform at constant number of particle, volume, and temperature (NVT) or the canonical ensemble. 

There are several advantages of MC methods. First, the perturbations can be performed in Cartersian or internal coordinates. Second, only energy evaluation is required and calculating derivatives of the energy is not needed. Third, because only one (or a few) particle is perturbed at each step, only the energy change associated with the perturbation needs to be calculated. Finally, it is easy to constrain certain degrees of freedom. One disadvantage of MC methods is its indepdence on time and velocities, and thus MC methods cannot explore time-depedent phenomena or properties that depend on the momentum.

MD methods explore the evolution of the system over time by integrating Newton's second law ($\textbf{F}=m \textbf{a}$). A small time step (typically $\sim 1$ fs) is usually used and therefore $10^6$ steps only explore the evolution of the system in 1 ns. Some physical processes, such as protein folding, occur at long time scales, and therefore short MD simulations only sample the region of the phase space close to the initial structure. Because the forces determine the *trajectory* of the particles, standard MD simulations cannot easily climb potential energy barriers because they will experience a force in the opposite direction. MD simulations are in principle deterministic, but very small numerical differences ($\sim 10^{-8}$) can rapidly cause the irreproducibility of MD trajectories. Because MD simulations have velocity and time dependence, they can explore time-dependent properties like transport phenomena. Unlike MC methods, MD simulations require the calculation of both the energy and its derivative. Furthermore, all the particles move each step, unlike MC methods. MD simulations are performed in Cartesian coordinates. Imposing constraints on the degrees of freedom is more challenging than in MC methods. For MD methods, the natural ensemble is the microcanonical ensemble (NVE; constant number of particles, volume, and energy), though other ensembles can also be generated. 

The differences between MC and MD methods are summarized below:
<img src="mc_vs_md.png" alt="drawing" width="600px"/>

## Monte Carlo
The basic idea of Monte Carlo methods is the random sampling of the phase space of molecules. As summarized above, a perurbation to the molecular structure is introduced either in Cartesian or internal coordinates. The move is always accepted if it lowers the energy. If it raises the energy, the move is accepted if $e^{-\Delta E/kT}$ is larger than a randomly generated number between 0 and 1.

### Random Number Generation
Typically, numbers are generated by a pseduo-random number generator, where the numbers are not truly random. The key point for random number generators is that they generate a very long sequence of apparently uncorrelated numbers. However, after a long sequence, the numbers will be repeated exactly. Usually, the sequence of (random) numbers can be fixed by using a given initial value (called *seed*). However, depending on the implementation, it may not be possible to generate the same sequence of numbers if different compilers or different computer architectures are used. Using an initial seed can be useful for debugging the code.

In [ ]:
# Numpy has different functionalities for random number generation
import numpy as np

# Generate an array of 6 random numbers
print("Random numbers 1:", np.random.rand(6))

# Generate a second array of 6 random numbers; They will be different
print("Random numbers 2:", np.random.rand(6))

# Now fix the seed using an integer value of zero
np.random.seed(0)

# Generate an array of 6 random numbers
print("Random numbers 1 (fixed seed):", np.random.rand(6))

# Now again set the seed value
np.random.seed(0)

# Generate a second array of 6 random numbers; The numbers will be the same
print("Random numbers 2 (same seed):", np.random.rand(6))

### Size of Perturbation Step
The size of the perturbation is an important parameter. A small size will lead to higher acceptance ratio, but it will lead to a slow exploration of the phase space. A large step will lead to a smaller acceptance ratio, and a sampling that consists of wide jumps.

### Nonphysical Moves
Because the perturbations are random, they can make physically unrealistic moves. This can be an advantage because this allows the Monte Carlo procedure to tunnel through high energy barrier. However, sometimes this can be a problem. For example, the chirality of the molecule may be inverted in a random move but this might be an undesired change.

### Correlated Motions
Because only one (or a few) particles are moved in each step, it is difficult to explore correlated changes in the configuration. For example, the dihedral angles in the protein backbone are correlated, and only certain combinations of values give an acceptable structure. However, a small perturbation in only one dihedral angle may raise the energy substantially because of atom clashes, and thus such a move will likely be rejected. 

## Molecular Dynamics
In molecular dynamics simulations, we use Newton's second law $\textbf{F} = m \textbf{a}$ to describe the evolution of the system over time. In the differential form, Newton's second law is given by:
\begin{align}
    -\frac{dV}{d\textbf{r}} = m \frac{d^2 \textbf{r}}{dt^2}
\end{align}
where $V$ is the potential energy at position $\textbf{r}$. Given an initial configuration $\textbf{r}_i$, we can determine the configuration at any other time by integrating Newton's equation. Except for a few simple cases, the integration cannot be performed analytically and therfore numerical integration is needed.

### Numerical Integrators
#### The Verlet Algorithm
The Verlet algorithm is a common approach for integrating Newton's equation. Given an initial configuration $\textbf{r}_i$, the position after a small time step $\Delta t$ is given by the Taylor expansion:
\begin{align}
    \textbf{r}_{i+1} &= \textbf{r}_i + \frac{\partial \textbf{r}}{\partial t} (\Delta t) + 
                              + \frac{1}{2} \frac{\partial^2 \textbf{r}}{\partial t^2} (\Delta t)^2 +
                              + \frac{1}{6} \frac{\partial^3 \textbf{r}}{\partial t^3} (\Delta t)^3 + \cdots\\
    \textbf{r}_{i+1} &= \textbf{r}_i + \textbf{v}_i (\Delta t) +  \frac{1}{2} \textbf{a}_i (\Delta t)^2 +
                        \frac{1}{6} \textbf{b}_i (\Delta t)^3 + \cdots
\end{align}
where $\textbf{v}_i, \textbf{a}_i, \textbf{b}_i$ are the velocity, acceleration, and hyper-acceleration at time $t_i$.

Similarly, the position at time step $\Delta t$ earlier is given by substituting $\Delta t$ with $-\Delta t$:
\begin{align}
    \textbf{r}_{i-1} &= \textbf{r}_i - \textbf{v}_i (\Delta t) +  \frac{1}{2} \textbf{a}_i (\Delta t)^2 -
                        \frac{1}{6} \textbf{b}_i (\Delta t)^3 + \cdots  
\end{align}

Addition of the two above equations gives a formula for calculating the position at time $\Delta t$ given the two previous positions and the current acceleration:
\begin{align}
    \textbf{r}_{i+1} &= (2 \textbf{r}_i - \textbf{r}_{i-1}) + \textbf{a}_i (\Delta t)^2 + \cdots \\
    \textbf{a}_i &= \frac{\textbf{F}_i}{m_i} = -\frac{1}{m_i} \frac{dV}{d \textbf{r}_i}.
\end{align}

This is the Verlet algorithm. The term involving $\textbf{b}$ disappears and therefore this expression is correct to third order in $\Delta t$. At the initial point, the previous position is estimated from the initial position and the initial velocity:
\begin{align}
    \textbf{r}_{-1} = \textbf{r}_0 - \textbf{v}_0 \Delta t
\end{align}

As the time step $\Delta t$ decreases, the approximation increases in accuracy. However, a small time step requires many simulation steps to reach a given total simulation time.

The velocities do not appear in the equations, which complicates calculation and the control of the temperature. The velocity can be estimated from the previous and next positions:
\begin{align}
    \textbf{v}_i = \frac{\textbf{r}_{i+1} - \textbf{r}_{i-1}}{2 \Delta t}.
\end{align}
This, however, has the numerical problem that the numerator involves the subtraction of two similar numbers divided by a small number. This can lead to numerical inaccuracies.

#### The Leap-frog Algorithm
The leap-frog algorithm solves some of the numerical problems of the Verlet algorithm and utilizes the velocities. Performing a similar expansion at a half time step later and subtracting gives:
\begin{align}
    \textbf{r}_{i + 1} = \textbf{r}_i + \textbf{v}_{i + \frac{1}{2}} \Delta t.
\end{align}
The velocity can be estimated using a similar expansion to get:
\begin{align}
    \textbf{v}_{i + \frac{1}{2}} = \textbf{v}_{i - \frac{1}{2}} + \textbf{a}_i \Delta t
\end{align}

Again, the expressions are correct to third order but it has better numerical accuracy than the Verlet algorithm. The disadvantage is that the position and velocity are not known at the same time. However, the explicit presence of the velocities allow for controling the temperature.

#### Velocity Verlet
In the velocity Verlet algorithm, both the velocity and the position are determined at the same time. The equations are given by:
\begin{align}
    \textbf{r}_{i+1} &= \textbf{r}_i + \textbf{v}_i \Delta t + \frac{1}{2} \textbf{a}_i (\Delta t)^2 \\
    \textbf{v}_{i+1} &= \textbf{v}_i + \frac{1}{2} ( \textbf{a}_i + \textbf{a}_{i+1}) \Delta t
\end{align}

#### Integration Time Step
The fastest process in the system determines the maximum time step that can be taken, with the appropriate time step typically being an order of magnitude slower. Molecular rotations and vibrations occur with frequencies in the range of $10^{11}-10^{14}$ s$^{-1}$. Thus, typically a 1 fs ($10^{-15}$) time step is usually chosen. A million time step therefore simulates the system for only 1 ns, which may not be enough to explore some chemical systems.

The fastest process is typically bond stretching that involves hydrogen atoms. These degrees of freedom, however, often have little influence on some chemical properties. Thus, bonds involving hydrogen atoms are often frozen using the *SHAKE* (Verlet), *LINCS* (leap-frog), or *RATTLE* (velocity verlet) algorithms. This allows a time step of 2 fs to be used. 

### Non-natural Ensembles
The natural ensemble for solving Newton's equation is the microcanonical ensemble (NVE). In this ensemble, the pressure and temperature fluctuate. The total energy is the sum of the kinetic and potential energies:
\begin{align}
    E_{\rm tot} = \sum_{i=1}^N \frac{1}{2} m_i \textbf{v}_i^2 + V(\textbf{r})
\end{align}

The average kinetic energy for a large number of particle $N$ is related to the temperature by
\begin{align}
    \langle E_{\rm kin} \rangle = \frac{3}{2} N k_{\rm B} T
\end{align}

The microcanonical ensemble is not convenient because it is difficult to devise experimental conditions where the NVE variables are constant. Experimentally, it is easier to fix the temperature and pressure. Ensembles with fixed termperature/pressure can also be generated for MD simulations

#### The Canonical Ensemble (NVT)
In the canonical ensemble, the temperature is fixed by coupling the system to a large heat reservior. Because the temperature is related to the kinetic energy and the velocity of the particles, the temperature of the simulation can be controlled by scaling the velocity of all particles. Scaling the velocity at each time step introduces unwanted changes in the dynamics. Instead, usually a *thermostat* or a heat bath is used to add or remove energy from the system at suitable time intervals. A commonly used thermostat is the Nose-Hoover thermostat.

#### The Isothermal-Isobaric Ensemble (NPT)
In addition to controlling the temperature, the pressure can also be controlled to produce the isothermal-isobaric ensemble (NPT). Again, a *barostat* or a pressure bath can be used to control the pressure. Barostats work by scaling the volume of the system.

#### Langevin and Brownian Dynamics
Sometimes, the interest is in the dynmaics of a small system in the presence of a large number of other particles. The forces in the system can be decomposed to three terms:friction
- A friction force proprtional to the velocity with a friction coefficient $\xi$ (Removes energy from the system).
- The intramolecular forces of the system
- A random force associated with the temperature and averages to zero (Add energy to the system).
With these forces, Newton's equation takes the form
\begin{align}
    m \frac{d^2 \textbf{r}}{dt^2} = -\xi \frac{d \textbf{r}}{dt} + \textbf{F}_{\rm intra} + \textbf{F}_{\rm random}
\end{align}
This is called the *Langevin* equation of motion and it is a stochastic equation. The limit that the friction force is dominant ($\xi$ is large) gives rise to *Brownian dynamics*. The Langevin integrator is often used to control the temperature in the simulation.

### Periodic Boundary Conditions
Molecular dynamics simulation are performed for relatively small systems. If the particles are not constrained somehow, some molecules may potentially boil off into space. Furthermore, for small systems, surface effects can become important. To address these limitations, *periodic boundary conditions* are usually used.

When periodic boundary conditions are employed in the simulation, a box of the system become surrounded by other fictitious simulation boxes as illustrated in the image below:
<img src="periodic_boundary_conditions.png" alt="drawing" width="200px"/>

If a particle escape from one side, then it re-appears in the opposite side because the system becomes periodic. A cubic box is often used but other types of boxes are also possible. Because electrostatic interactions is a long-range force in the system, it needs special treatment for periodic systems. Electrostatic interactions for periodic systems are usually calculated using the Ewald sum methods discussed before. 

### Extracting Information from Simulations
The main output of an MD simulation is the MD trajectory, which is the sequence of positions (and possibly momenta) of the particles at each time step. Furthermore, other properties, such as thermodynamic quantities, can be extracted from the simulations. We generally calculated *average* quantities from MD trajectories. A single MD snapshot does not represent a statistically meaningful observation.

#### The Ergodic Hypothesis
In statistical mechanics, we are concerned with *average* properties. A single particle (or a few particles) explores the phase space over time, and thus we can define an *average over time* as:
\begin{align}
    \langle X \rangle = \lim_{\tau \to \infty} \frac{1}{\tau} \int_0^\tau X(t) dt
\end{align}
Alternatively, for a large number of identical particles, each particle will occupy a certain region of the phase space at any given time. Thus, we can also calculate the *ensemble average* as:
\begin{align}
    \langle X \rangle = \lim_{M \to \infty} \frac{1}{M} \sum_{i=1}^M X_i
\end{align}

The *ergodic hypothesis* asserts that these two averages are equal.

#### Equilibration and Production Stages
To perform an MD simulation, an initial structure is required. This is often an X-ray structure of the system if available or some guess structure generated by a modeling program. This initial structure is usually high in energy. To prevent the presence of strong forces in the beginning which can destabilize the simulation, energy minimization to a nearby local minima is performed before running the simulation. Still, the structure in the beginning of the simulation can still be far from equilibrium.

The above equations for the averages assume that the system is at equilibrium. Therefore, an MD trajectory is usually divided to two stages: *equilibration* and *production*. The equilibration stage is supposed to bring the system to the equilibrium structure. Data from the equilibration stage are not used to calculate the averages. Only data from the production stage are used.

There is no strict way to partition the trajectory to equilibration and production stages. Often, some initial portion of the trajectory is discarded as equilibration. Discarding too little can bias the results towards the initial structure while discarding too much can increase the statistical error in the average data. Generally, thermodynamic properties, such as the energy and the temperature, can be monitored to establish whether they reached stable values with limited fluctuations.

#### Uncorrelated Samples
The above averages assume that the samples are *uncorrelated*. In MD simulations, snapshots should be saved at suitable intervals of time where the property of interest is uncorrelated. Saving too many snapshots can take a lot of storage space while adding little statistical significance. Different ways can be used to estimate the correlation times for different properties.

#### Some Properties

##### Temperature
The temperature can be calculated from the average kinetic energy as indicated above

##### Internal Energy
The internal energy is just calculated from the force field energy at each time step. It will typically fluctuate around an average value.

##### Heat Capacity
The heat capacity at constant volume is the derivative of the energy with respect to temperature at constant volume:
\begin{align}
    C_V = \left ( \frac{\partial U}{\partial T} \right )_V
\end{align}
It can be estimated by running the simulation at different temperatures, estimating $\langle U \rangle$ as a function of temperature, and taking its derivative. However, this is inconvenient because it requires multiple simulations.

The heat capacity can also be obtained from the fluctuation in the internal energy as
\begin{align}
    C_V = \frac{1}{kT^2} ( \langle U^2 \rangle - \langle U \rangle^2)
\end{align}
The correlation time for the fluctuations is generally larger than the correlation time for the internal energy, and thus longer simulations may be required to converge the heat capacity

##### Radial Distribution Function
The radial distribution function $g(r)$ measures the relative probability of finding a reference particle as a function of the distance from other particles. The probability is relative to a completely uniform distribution, which has a density of $N/V$. The figure below illustrates the calculation of $g(r)$:
<img src="rdf.png" alt="drawing" width="200px"/>

A typical behavior of the radial distribution function is shown below:
<img src="rdf_plot.png" alt="drawing" width="500px"/>

$g(r)$ starts at zero because the size of the reference particle excludes other particles. The first peak correseponds to the first solvation shell, and the second peak corresponds to the second solvation shell, etc. $g(r)$ levels off to 1 because this is the relative probability for a uniform distribution of particles. 

### Free Energy Methods
Various methods can be used to estimate the free energy difference between two states, e.g. Helmholtz or Gibbs free energy difference. A key point about the free energy difference is that it is a *state function*. That is, it does not depend on the path that connects the two states. This allows molecular simulation to use any path, even physically impossible paths, to estimate the difference between the two states.

#### Free Energy Perturbation
The free energy difference between states A and B can be written as an ensemble average
\begin{align}
    \langle \Delta A_{\rm AB} \rangle = k T {\rm ln} \langle e^{(E_{\rm B} - E_{\rm A})/kT}\rangle
\end{align}
In practice, to converge the free energy difference, the states A and B must be sufficiently similar. To facilitate this, the energy of the system in terms of a parameter $\lambda$ that connects the two states A and B:
\begin{align}
    E_\lambda = \lambda E_{\rm A} + (1-\lambda) E_{\rm B}.
\end{align}
where $\lambda$ is from zero to one. Then, the energy difference is calculated for different values of $\lambda$.

To validate the calculated free energies, the transformation is usually carried out in both directions (A $\rightarrow$ B and B $\rightarrow$ A). The difference in energy between the two transformations gives an estimate of the convergence of the free energy.

#### Thermodynamic Integration
In thermodynamic integration, the energy of the system is again written in terms of the parameter $\lambda$. The free energy difference is then estimated as
\begin{align}
    \Delta A = \sum_i \left \langle \frac{\partial E(\lambda)}{\partial \lambda} \right \rangle d \lambda_i
\end{align}
where again different values of $\lambda$ are used.

#### Umbrella Sampling
In umbrella sampling, a bias is introduced in order to allow the system to overcome potential energy barrier along some reaction coordinate or degree of freedom of the system. Then, the unbiased distribution is estimated from the biased simulations and the free energy is calculated. The free energy along the chosen coordinate is called the *potential of mean force*. 

## Useful Resources

- Cramer, C. J. *Essentials of Computational Chemistry: Theories and Models*, 2nd ed.; John Wiley & Sons: Chichester, England, 2004. (Chapters 3, 12, and 13)
- Jensen, F. *Introduction to Computational Chemistry*, 3rd ed.; John Wiley & Sons: Nashville, TN, 2017. (Chapter 15)